In [29]:
import numpy as np
import pandas as pd
import os, glob
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# Load Standard Scaler (same as z-normalization) for normalization of whole image
scaler = StandardScaler()

In [30]:
# Call subjects from desired institution(this code only works for one institutions)
seg_dir = 'segmentation images directory'
baseline_dir = 'baseline images(t1,t1ce,t2,flair,adc)'
near_dir = 'randomly selected near ROI coordinates csv file directory'
far_dir = 'randomly selected far ROI coordinates csv file directory'

segm_dir = sorted(glob.glob(os.path.join(seg_dir, "*_segm*.nii.gz")))

flair_dir = sorted(glob.glob(os.path.join(baseline_dir, "*_flair*.nii.gz"))) 
t1_dir = sorted(glob.glob(os.path.join(baseline_dir, "*_t1_*.nii.gz"))) 
t1ce_dir = sorted(glob.glob(os.path.join(baseline_dir, "*_t1ce*.nii.gz"))) 
t2_dir = sorted(glob.glob(os.path.join(baseline_dir, "*_t2*.nii.gz"))) 
adc_dir = sorted(glob.glob(os.path.join(baseline_dir, "*_adc*.nii.gz"))) 

nearSample_dir = sorted(glob.glob(os.path.join(near_dir, "*_near*.csv")))
farSample_dir = sorted(glob.glob(os.path.join(far_dir, "*_far*.csv")))

In [31]:
# Read csv file and put into panda dataframe
def load_coords_from_csv(file_path):
    df = pd.read_csv(file_path)
    coords = df.to_numpy()
    return coords

In [32]:
# Load nifti files
def LoadingImage(dir):
    
    nifti_image = nib.load(dir)
    image = np.asarray(nifti_image.dataobj)
    header = nifti_image.header
    imgaffine = nifti_image.affine
    
    return image, header, imgaffine

In [33]:
# Separately collects unique subject name from files name
def getFilename(full_dir):
    _,filename = full_dir.split('\\')
    print(filename)
    subject,_,_,_,_= filename.split('_')
    return str(subject)

In [34]:
# Create 5x5x5 voxels patches where selected voxel is center of each patch
def extract_patch(image, center, patch_size):
    x, y, z = center
    x_size, y_size, z_size = patch_size

    x_start, y_start, z_start = x - x_size // 2, y - y_size // 2, z - z_size // 2
    x_end, y_end, z_end = x_start + x_size, y_start + y_size, z_start + z_size

    return image[x_start:x_end, y_start:y_end, z_start:z_end]

In [ ]:
for i in range(len(segm_dir)):

    # call seg the images
    seg_img, seg_hdr, seg_imgaffine = LoadingImage(segm_dir[i])
    # call flair image (240, 240, 155)
    flair_img, flair_hdr, flair_imgaffine = LoadingImage(flair_dir[i])
    flair_img = scaler.fit_transform(flair_img.reshape(-1, 1)).reshape(flair_img.shape)
    # call t1 image (240, 240, 155)
    t1_img, t1_hdr, t1_imgaffine = LoadingImage(t1_dir[i])
    t1_img = scaler.fit_transform(t1_img.reshape(-1, 1)).reshape(t1_img.shape)
    # call t1ce image (240, 240, 155)
    t1ce_img, t1ce_hdr, t1ce_imgaffine = LoadingImage(t1ce_dir[i])
    t1ce_img = scaler.fit_transform(t1ce_img.reshape(-1, 1)).reshape(t1ce_img.shape)
    # call t2 image (240, 240, 155)
    t2_img, t2_hdr, t2_imgaffine = LoadingImage(t2_dir[i])
    t2_img = scaler.fit_transform(t2_img.reshape(-1, 1)).reshape(t2_img.shape)
    # call adc image (240, 240, 155)
    adc_img, adc_hdr, adc_imgaffine = LoadingImage(adc_dir[i])
    adc_img = scaler.fit_transform(adc_img.reshape(-1, 1)).reshape(adc_img.shape)

    nearCoords = load_coords_from_csv(nearSample_dir[i])
    farCoords = load_coords_from_csv(farSample_dir[i])

    # Set patch size
    patch_size = (5, 5, 5)

    # filename
    filename = getFilename(segm_dir[i])

    # Iterate over the coordinates, extract patches, and save them as NIfTI files
    dir_path = 'desired ouput space for save patches for near ROI'
    k = 0
    for coord in nearCoords:
        
        flair_patch = extract_patch(flair_img, coord, patch_size)
        flair_patch_img = nib.Nifti1Image(flair_patch, flair_imgaffine, flair_hdr)
        nib.save(flair_patch_img, dir_path + f'{filename}_flairPatch_{k}.nii.gz')

        t1_patch = extract_patch(t1_img, coord, patch_size)
        t1_patch_img = nib.Nifti1Image(t1_patch, t1_imgaffine, t1_hdr)
        nib.save(t1_patch_img, dir_path + f'{filename}_t1Patch_{k}.nii.gz')

        t1ce_patch = extract_patch(t1ce_img, coord, patch_size)
        t1ce_patch_img = nib.Nifti1Image(t1ce_patch, t1ce_imgaffine, t1ce_hdr)
        nib.save(t1ce_patch_img, dir_path + f'{filename}_t1cePatch_{k}.nii.gz')

        t2_patch = extract_patch(t2_img, coord, patch_size)
        t2_patch_img = nib.Nifti1Image(t2_patch, t2_imgaffine, t2_hdr)
        nib.save(t2_patch_img, dir_path + f'{filename}_t2Patch_{k}.nii.gz')

        adc_patch = extract_patch(adc_img, coord, patch_size)
        adc_patch_img = nib.Nifti1Image(adc_patch, adc_imgaffine, adc_hdr)
        nib.save(adc_patch_img, dir_path + f'{filename}_adcPatch_{k}.nii.gz')

        k = k+1
    
    # Iterate over the coordinates, extract patches, and save them as NIfTI files
    dir_path = 'desired ouput space for save patches for far ROI'
    k = 0
    for coord in farCoords:
        
        flair_patch = extract_patch(flair_img, coord, patch_size)
        flair_patch_img = nib.Nifti1Image(flair_patch, flair_imgaffine, flair_hdr)
        nib.save(flair_patch_img, dir_path + f'{filename}_flairPatch_{k}.nii.gz')

        t1_patch = extract_patch(t1_img, coord, patch_size)
        t1_patch_img = nib.Nifti1Image(t1_patch, t1_imgaffine, t1_hdr)
        nib.save(t1_patch_img, dir_path + f'{filename}_t1Patch_{k}.nii.gz')

        t1ce_patch = extract_patch(t1ce_img, coord, patch_size)
        t1ce_patch_img = nib.Nifti1Image(t1ce_patch, t1ce_imgaffine, t1ce_hdr)
        nib.save(t1ce_patch_img, dir_path + f'{filename}_t1cePatch_{k}.nii.gz')

        t2_patch = extract_patch(t2_img, coord, patch_size)
        t2_patch_img = nib.Nifti1Image(t2_patch, t2_imgaffine, t2_hdr)
        nib.save(t2_patch_img, dir_path + f'{filename}_t2Patch_{k}.nii.gz')

        adc_patch = extract_patch(adc_img, coord, patch_size)
        adc_patch_img = nib.Nifti1Image(adc_patch, adc_imgaffine, adc_hdr)
        nib.save(adc_patch_img, dir_path + f'{filename}_adcPatch_{k}.nii.gz')

        k = k+1